# Education EDA
This notebook explores educational assessment data provided from NCES. The original datasets were achievement broken down by subject, Local Educational Agency (LEA), and school year. For now, we'll focus on 2017-18.

1. [Import Data](#Import-Data)

2. [Preview Data and Structure](#Preview-Data)

3.  [MATH ASSESSMENT](#Math-Assessment-Data)

4. [Reading & Language Arts (RLA) Assessment](#Reading-and-Language-Arts-Assessment)

[Census Broadband Dataa](#Census-Broadband-Data)

[Census Device Data](#Census-Device-Data)

[Geographical Data](#Geographical-Data)


# Import Data

In [1]:
## Import necessary modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from itertools import compress

## Functions to call data from website

### Links

[Math Achievement Percentage Proficiency by Local Education Agency, 2017-18](https://www2.ed.gov/about/inits/ed/edfacts/data-files/math-achievement-lea-sy2017-18.csv)

[Reading / Language Arts Achievement Percentage Proficiency by Local Education Agency, 2017-18](https://www2.ed.gov/about/inits/ed/edfacts/data-files/rla-achievement-lea-sy2017-18.csv)

In [ ]:
## Import 2018 LEA  math data to start

# subject=input("Enter Assessment Subject: 'math' for math OR 'rla' for reading / language arts")
# geog=input("Enter Detail Level: 'lea' for Local Education Agency OR 'sch' for school")
# data_file=input("Enter dataset type: 'achievement' for assessment proficiency OR 'participation' for assessment participation OR 'acgr' for Adjusted Cohort Graduation Rate")
# school_year=input("Enter academic year as: YYYY-YY")

In [ ]:
subject='math'
data_file='achievement'
geog='lea'
school_year='2017-18'

math_2018 = pd.read_csv('https://www2.ed.gov/about/inits/ed/edfacts/data-files/{subject}-{data_file}-{geog}-sy{school_year}.csv'
                 .format(subject=subject, data_file=data_file,  geog=geog, school_year=school_year),
                 low_memory=False)
math_2018.rename(columns={'LEAID':'leaid'}, inplace=True)
math_2018['leaid'] = math_2018['leaid'].astype(str)
math_2018['leaid'] = math_2018['leaid'].str.lstrip('9700000US').str.zfill(7)
math_2018.to_pickle('../data/{subject}_{data_file}_{school_year}.pkl'.format(subject=subject, data_file=data_file, school_year=school_year))
math_2018.head()

In [ ]:
## Also Import RLA 2018 data

# subject=input("Enter Assessment Subject: 'math' for math OR 'rla' for reading / language arts")
# geog=input("Enter Detail Level: 'lea' for Local Education Agency OR 'sch' for school")
# data_file=input("Enter dataset type: 'achievement' for assessment proficiency OR 'participation' for assessment participation OR 'acgr' for Adjusted Cohort Graduation Rate")
# school_year=input("Enter academic year as: YYYY-YY")

In [ ]:
subject='rla'
data_file='achievement'
geog='lea'
school_year='2017-18'

rla_2018 = pd.read_csv('https://www2.ed.gov/about/inits/ed/edfacts/data-files/{subject}-{data_file}-{geog}-sy{school_year}.csv'
                 .format(subject=subject, data_file=data_file,  geog=geog, school_year=school_year),
                 low_memory=False)
rla_2018.rename(columns={'LEAID':'leaid'}, inplace=True)
rla_2018['leaid'] = rla_2018['leaid'].astype(str)
rla_2018['leaid'] = rla_2018['leaid'].str.lstrip('9700000US').str.zfill(7)
rla_2018.to_pickle('../data/{subject}_{data_file}_{school_year}.pkl'.format(subject=subject, data_file=data_file, school_year=school_year))
rla_2018.head()

In [ ]:
assessment_2018 = pd.merge(math_2018, rla_2018, how='inner', on='leaid')
assessment_2018.to_pickle('../data/assessment_2018.pkl')
assessment_2018.info()

In [ ]:
assessment_2018.shape

In [ ]:
census_2018 = pd.read_pickle('ACS_5yr_DP')

In [ ]:
math_2018.shape

#### Rows
There are almost 16,000 rows here. They appear to be unique observations for each Local Education Agency (LEA), but let's confirm:

In [ ]:
## Ensure that LEAID is a unique value for each observation
math_2018['LEAID'].nunique() == math_2018.shape[0]
## Returns True if all values are unique

Great. Since the unique number of LEAIDs (15984) matches the total number of observations for the dataset, we know that it is a unique value. We can use this value as our index later.

#### Columns
To preview what's in them better, we'll look at just the columns to see if we can eliminate redundancies/noise.

The first set of columns contains identifying data:
- STNAME - State Name
- FIPST - ANSI 2-digit code for state
- LEAID - NCES-assigned District ID
- ST_LEAID - State-assigned District ID
- LEANM - District Name
- DATE_CUR - Date of data snapshot

And the rest contain various groupings of the data for each observation (LEA) in the following format:
- \[SUBGROUP\]_\[SUBJECT\]\[GRADE\]\[METRIC\]_\[SCHOOLYEAR\]

Reading the [documentation]('..\data\education\education_documentation_2018.docx) provides an overview of those columns, along with their corresponding meanings:

- **Subgroups** - Major Racial and Ethnic Groups / Special Populations
    - ALL - All Students in the school
    - Racial Subgroups (begin with M):
        - MAM American Indian/Alaska Native
        - MAS Asian/Pacific Islander
        - MHI Hispanic
        - MBL Black
        - MWH White
        - MTR Two or More Races
    - Special Population Subgroups:
        - CWD Children with disabilities
        - ECD Economically disadvantaged
        - LEP Limited English proficiency
        - HOM Homeless
        - MIG Migrant
        - FCS Foster Care Status
        - MIL Military Connected (new for '17-'18)
        
- **Subject** - Specific to each file.
    - Math (MTH) or Reading/Language Arts (RLA)
    
- **Grade** 
    - 00 - aggregated across all grades
    - 03-08 - Grades 3-8
    - HS - High School
    
- **Metric**
    - numvalid - # of students who completed assessment proficient
    - pctprof - % of students proficient or higher
    
- **School Year**
    - Limited to 2017-18 in this case
    - May be able to add additional years later for better understanding
    
So, for example, the column labeled "**MBL_MTH08PCTPROF_1718**" describes: 
- in 2017-18 **\[1718\]**,
- for Black students **\[MBL\]** 
- studying Math **\[MTH\]**
- in the Eighth grade **\[08\]**, 
- what Percentage were deemed Proficient **\[PCTPROF\]**



Knowing and observing this breakdown might prove helpful in improving the model later. For now, however, we'll keep only the following columns:
- LEAID - as our unique identifier / index. We'll convert this to a 7-digit string for concatenation.
- ALL_MTHHSPCTPROF_1718 - as our prediction column.

Given the business understanding and limitation to HS students, this makes sense. Since we don't have a total number of students who took the exam, percentage proficient will standardize the number proficient across districts.

## Math Assessment Data

In [ ]:
a
    
#     elif "GT" or "GE" in score:
#         if "GT" in score:
#             score = score.strip("GT")
#             score = (100 - int(score))/2
#             return round(int((score)))

#         elif "GE" in score:
#             score = score.strip("GE")
#             score = (100 - int(score))/2
#             return round(int((score)))
    
#     elif "LT" or "LE" in score:
#         if "LT" in score:
#             score = score.strip("LT")
#             score = (0 + int(score))/2
#             return round(int((score)))

#         elif "LE" in score:
#             score = score.strip("LE")
#             score = (0 + int(score))/2
#             return round(int((score)))
    else:
        return int(score)

In [ ]:
## Create new dataframe with just the two desired columns
all_math_2018 = math_2018[['LEAID','ALL_MTHHSPCTPROF_1718']].dropna()
all_math_2018.columns = all_math_2018.columns.str.lower()
all_math_2018.rename(columns={'all_mthhspctprof_1718': 'math_score'}, inplace=True)
all_math_2018.leaid = all_math_2018.leaid.apply(str)
all_math_2018.math_score = all_math_2018.math_score.apply(str)
all_math_2018.leaid = all_math_2018.leaid.str.zfill(7)
all_math_2018 = pd.DataFrame(data=all_math_2018)
print(all_math_2018.head())
print(all_math_2018.info())

In [ ]:
PS_values = all_math_2018[all_math_2018['math_score'] == "PS" ].index
period_values = all_math_2018[all_math_2018['math_score'] == "." ].index
LE_values = all_math_2018[all_math_2018.math_score.str.contains("LE")].index
GT_values = all_math_2018[all_math_2018.math_score.str.contains("GE")].index
GE_values = all_math_2018[all_math_2018.math_score.str.contains("GT")].index
LT_values = all_math_2018[all_math_2018.math_score.str.contains("LT")].index

all_math_2018.drop(PS_values, inplace = True)
all_math_2018.drop(period_values, inplace = True)
all_math_2018.drop(LT_values, inplace = True)
all_math_2018.drop(LE_values, inplace = True)
all_math_2018.drop(GT_values, inplace = True)
all_math_2018.drop(GE_values, inplace = True)
all_math_2018.info()

In [ ]:
all_math_2018.math_score = all_math_2018.math_score.apply(score_cleaner)
all_math_2018.info()

In [ ]:
# all_math_2018.math_score = all_math_2018.loc[~all_math_2018.math_score.str.contains("LT"),:]
# all_math_2018.dropna(inplace=True)
# all_math_2018.math_score = all_math_2018.loc[~all_math_2018.math_score.str.contains("GE"),:]
# all_math_2018.dropna(inplace=True)
# all_math_2018.math_score = all_math_2018.loc[~all_math_2018.math_score.str.contains("GT"),:]
# all_math_2018.dropna(inplace=True)

# all_math_2018.head()

This dataset is much cleaner. 

However, looking closer at the target variable column we have two issues in the target variable column:
1. Null values - around 3600
2. Datatype - is classified as object

Not only is the target variable classified as an object, but looking at the 5th observation, the objects do not appear to be readily convertible to integers, since some of them appear to be ranges. Taking a further look:

In [ ]:
all_math_2018['math_score'].unique()

Further inspection reveals that "nan" values, "." values (also described as blank in documentation),  ranges, and strings are complicating the numerical analysis. Referring again to the documentation, these variations were intentionally introduced for student privacy concerns, with the following codes:

- GE - Greater than or equal to
- LE - Less than or equal to
- GT Greater than
- LT - Less than
- PS - Privacy Suppressed (<5 students)

Specifically, as specified in Part 1.5 and Table 2 of the documentation for 2017-18, for schools with less than 300 students, "As the number of students reported increases, the magnitude of the range decreases". 

Given these ranges, we can make a few choices:

1. Limit the study to large high schools (300+). 
    - Benefit: Leaves only whole numbers, enabling integer conversion. 
    - Benefit: Allows for the possible usage of linear regression since it would be a continuous variable
    - Cost: Eliminates the potential for analyzing size as a factor; Unnecessarily eliminates data
    

2. Bin the score ranges...more options here: 
    - Smaller scale (e.g., 25-29)
    - Medium-scale (e.g., Very High proficiency (>80%), high proficiency (60-80), proficient (40-60), low proficiency (20-40), very low proficiency (0-20)
    - Large scale (e.g., >= 50% as proficient)
    

For now, let's focus on large high schools. In this dataset, this means limiting the observations selected to those with 1 or 2 characters that are not "PS":

In [ ]:
all_math_2018.hist()

Unfortunately, using only the schools that hadn't been "blurred" will result in too few datapoints. For now, though, let's continue by repeating this procedure for Reading and Language Arts data (RLA).

## Reading and Language Arts Assessment

In [ ]:
rla_2018 = pd.read_csv('../data/education/rla-achievement-lea-sy2017-18.csv', low_memory=False)

## Create new dataframe with just the two desired columns
all_rla_2018 = rla_2018[['LEAID','ALL_RLAHSPCTPROF_1718']].dropna()
all_rla_2018.columns = all_rla_2018.columns.str.lower()
all_rla_2018.rename(columns={'all_rlahspctprof_1718': 'rla_score'}, inplace=True)
all_rla_2018.leaid = all_rla_2018.leaid.apply(str)
all_rla_2018.rla_score = all_rla_2018.rla_score.apply(str)
all_rla_2018.leaid = all_rla_2018.leaid.str.zfill(7)
all_rla_2018 = pd.DataFrame(data=all_rla_2018)
print(all_rla_2018.head())
print(all_rla_2018.info())

In [ ]:
rla_2018.head()

In [ ]:
PS_values = all_rla_2018[all_rla_2018['rla_score'] == "PS" ].index
period_values = all_rla_2018[all_rla_2018['rla_score'] == "." ].index
LE_values = all_rla_2018[all_rla_2018.rla_score.str.contains("LE")].index
GT_values = all_rla_2018[all_rla_2018.rla_score.str.contains("GE")].index
GE_values = all_rla_2018[all_rla_2018.rla_score.str.contains("GT")].index
LT_values = all_rla_2018[all_rla_2018.rla_score.str.contains("LT")].index

all_rla_2018.drop(PS_values, inplace = True)
all_rla_2018.drop(period_values, inplace = True)
all_rla_2018.drop(LT_values, inplace = True)
all_rla_2018.drop(LE_values, inplace = True)
all_rla_2018.drop(GT_values, inplace = True)
all_rla_2018.drop(GE_values, inplace = True)
all_rla_2018.info()

In [ ]:
all_rla_2018.rla_score = all_rla_2018.rla_score.apply(score_cleaner)
all_rla_2018.info()

In [ ]:
all_rla_2018.head()

In [ ]:
hs_assessments_2018 = pd.merge(all_rla_2018, all_math_2018, how='inner', on='leaid').dropna()
hs_assessments_2018['year'] = 2018
hs_assessments_2018.info

In [ ]:
hs_assessments_2018.to_pickle('../data/education/hs_assessments_2018.pkl')

In [ ]:
sns.pairplot(assessments_2018, vars=['math_score', 'rla_score'], kind='reg', diag_kind='kde');

As is probably expected, there is a definite correlation between Reading/Language Arts Scores and Math Scores for the limited dataset we're observing. It should be noted that the math score distribution skews slightly to the left. The RLA score skews to the right. This phenomenon is confirmed in the statistical summary which shows us that the mean math score is 45% and the mean reading/language arts score is 55%.

### K-12

#### k-12 math

In [ ]:
whole_math_2018 = math_2018[['LEAID','ALL_MTH00PCTPROF_1718']].dropna()
whole_math_2018.columns = whole_math_2018.columns.str.lower()
whole_math_2018.rename(columns={'all_mth00pctprof_1718': 'math_score'}, inplace=True)
whole_math_2018.leaid = whole_math_2018.leaid.apply(str)
whole_math_2018.math_score = whole_math_2018.math_score.apply(str)
whole_math_2018.leaid = whole_math_2018.leaid.str.zfill(7)
whole_math_2018 = pd.DataFrame(data=whole_math_2018)
print(whole_math_2018.head())
print(whole_math_2018.info())

In [ ]:
PS_values = whole_math_2018[whole_math_2018['math_score'] == "PS" ].index
period_values = whole_math_2018[whole_math_2018['math_score'] == "." ].index
LE_values = whole_math_2018[whole_math_2018.math_score.str.contains("LE")].index
GT_values = whole_math_2018[whole_math_2018.math_score.str.contains("GE")].index
GE_values = whole_math_2018[whole_math_2018.math_score.str.contains("GT")].index
LT_values = whole_math_2018[whole_math_2018.math_score.str.contains("LT")].index

whole_math_2018.drop(PS_values, inplace = True)
whole_math_2018.drop(period_values, inplace = True)
whole_math_2018.drop(LT_values, inplace = True)
whole_math_2018.drop(LE_values, inplace = True)
whole_math_2018.drop(GT_values, inplace = True)
whole_math_2018.drop(GE_values, inplace = True)
whole_math_2018.info()

In [ ]:
whole_math_2018.math_score = whole_math_2018.math_score.apply(score_cleaner)
whole_math_2018.info()

In [ ]:
whole_math_2018.to_pickle('../data/education/whole_math_2018.pkl')

#### k-12 rla

In [ ]:
## Create new dataframe with just the two desired columns

whole_rla_2018 = rla_2018[['LEAID','ALL_RLA00PCTPROF_1718']].dropna()
whole_rla_2018.columns = whole_rla_2018.columns.str.lower()
whole_rla_2018.rename(columns={'all_rla00pctprof_1718': 'rla_score'}, inplace=True)
whole_rla_2018.leaid = whole_rla_2018.leaid.apply(str)
whole_rla_2018.rla_score = whole_rla_2018.rla_score.apply(str)
whole_rla_2018.leaid = whole_rla_2018.leaid.str.zfill(7)
whole_rla_2018 = pd.DataFrame(data=whole_rla_2018)
print(whole_rla_2018.head())
print(whole_rla_2018.info())

In [ ]:
PS_values = whole_rla_2018[whole_rla_2018['rla_score'] == "PS" ].index
period_values = whole_rla_2018[whole_rla_2018['rla_score'] == "." ].index
LE_values = whole_rla_2018[whole_rla_2018.rla_score.str.contains("LE")].index
GT_values = whole_rla_2018[whole_rla_2018.rla_score.str.contains("GE")].index
GE_values = whole_rla_2018[whole_rla_2018.rla_score.str.contains("GT")].index
LT_values = whole_rla_2018[whole_rla_2018.rla_score.str.contains("LT")].index

whole_rla_2018.drop(PS_values, inplace = True)
whole_rla_2018.drop(period_values, inplace = True)
whole_rla_2018.drop(LT_values, inplace = True)
whole_rla_2018.drop(LE_values, inplace = True)
whole_rla_2018.drop(GT_values, inplace = True)
whole_rla_2018.drop(GE_values, inplace = True)
whole_rla_2018.info()

In [ ]:
whole_rla_2018.rla_score = whole_rla_2018.rla_score.apply(score_cleaner)
whole_rla_2018.info()

In [ ]:
whole_rla_2018.to_pickle('../data/education/whole_rla_2018.pkl')

#### k-12 combined

In [ ]:
combined_assessments_2018 = pd.merge(whole_rla_2018, whole_math_2018, how='inner', on='leaid').dropna()
combined_assessments_2018['year'] = 2018
combined_assessments_2018.info

In [ ]:
combined_assessments_2018.to_pickle('../data/education/combined_assessments_2018.pkl')

## Census Broadband Data

## Census Device Data

## Geographical Data

## Census Other Features

In [ ]:
acs5_2018 = pd.read_pickle('../data/digital/acs5_2018.pkl')

In [ ]:
full_working_set = pd.read_pickle('../data/full_working_set.pkl')

In [ ]:
complete_df = pd.read_pickle('../data/complete_df.pkl')
complete_df.head()